In [88]:
import pandas as pd


df = pd.read_csv('data/normalized_filtered_user_listening.csv', usecols=lambda column: column not in ['Unnamed: 0'])

In [89]:
df['normalized_playcount'].describe()

count    3.651141e+06
mean     4.704546e-01
std      3.569481e-01
min      5.373455e-04
25%      1.538462e-01
50%      3.333333e-01
75%      9.000000e-01
max      1.000000e+00
Name: normalized_playcount, dtype: float64

### Surprise library does not use sparse matrix, thus number of data taken is limited

In [90]:
column_order=['user_id','track_id','normalized_playcount']
df=df[column_order]
small_df=df[0:10000]
small_df

,user_id,track_id,normalized_playcount
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,TRAAHSY128F147BB5C,1.000000
1,85c1f87fea955d09b4bec2e36aee110927aedf9a,TRPGYLT128F428AD02,1.000000
2,bd4c6e843f00bd476847fb75c47b4fb430a06856,TRWCEKE128F93191BE,1.000000
3,4bd88bfb25263a75bbdd467e74018f4ae570e5df,TRDSFKT12903CB510F,0.500000
4,4bd88bfb25263a75bbdd467e74018f4ae570e5df,TRRELZC128E078ED67,1.000000
...,...,...,...
9995,5968a59e582f434a223b3786cd51c9f4690b38d4,TRDAEMU128F92C5A76,1.000000
9996,2568aff0ee8deecab033c40a8198efd39bfd2b38,TRZGTZF12903CC562D,0.153846
9997,2568aff0ee8deecab033c40a8198efd39bfd2b38,TRWSJPN12903CB2CC6,0.076923
9998,2568aff0ee8deecab033c40a8198efd39bfd2b38,TRFHJOI128EF34BFAF,0.615385


In [25]:
from surprise import accuracy, Dataset, SVD
from surprise.model_selection import train_test_split
from surprise import Dataset, NormalPredictor, Reader

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(0, 1))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(small_df[["user_id", "track_id", "normalized_playcount"]], reader)

# sample random trainset and testset
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=0.25)

# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 0.3389


0.3388958539455192

In [33]:
[testset[0]]

[('cedacf9603a3c6c4cd1fd869b4572232664a1fb2', 'TRCRCBT128F4260DD1', 1.0)]

In [34]:
predictions = algo.test([testset[0]])

In [35]:
predictions

[Prediction(uid='cedacf9603a3c6c4cd1fd869b4572232664a1fb2', iid='TRCRCBT128F4260DD1', r_ui=1.0, est=0.32602248359568464, details={'was_impossible': False})]

In [49]:
from surprise import accuracy, Dataset, SVD
from surprise.model_selection import train_test_split
from surprise import Dataset, NormalPredictor, Reader
from surprise import KNNBaseline

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(0, 1))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(small_df[["user_id", "track_id", "normalized_playcount"]], reader)

# sample random trainset and testset
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=0.25)

# We'll use the famous SVD algorithm.
algo = KNNBaseline()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.3475


0.3475297309202985